In [1]:
import csv
import math
import cv2
from PIL import Image
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import Model
from tensorflow.keras import layers,models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions


Using TensorFlow backend.


In [2]:
import tensorflow
tensorflow.__version__
import keras
keras.__version__

'2.2.4'

In [23]:
ALPHA = 0.35

# 96, 128, 160, 192, 224
IMAGE_SIZE = 96
EPOCHS = 7
BATCH_SIZE = 32
PATIENCE = 50
MULTI_PROCESSING = True
THREADS = 4

In [4]:

TRAIN_CSV = "C:/Users/itsab/Downloads/training_set.csv"  #img_dir='drive/inter_iit/conv_data/sat_match2'
#VALIDATION_CSV = "C:/Users/itsab/Downloads/validation_try_1.csv"
path2="C:/Users/itsab/Desktop/Flipkart/images/"
model_path="C:/Users/itsab/Desktop/Flipkart/model_try_3_1.hd5"


In [24]:
import PIL
class DataGenerator(Sequence):

    def __init__(self, csv_file):
        self.paths = []
        
        with open(csv_file, "r") as file:
            self.coords = np.zeros((sum(1 for line in file), 4))
            print(self.coords.shape)
            file.seek(0)

            reader = csv.reader(file, delimiter=",")
            real_index=0;
            for index, row in enumerate(reader):
                #for i, r in enumerate(row[1:5]):
                    #row[i+1] = int(r)
              if not index==0: 
                try:    
                  path=row[0];

                  path=path2+path;
                  filename=path
                  im = Image.open(filename)
                  im.verify() 
                  im.close() 
                  im = Image.open(filename) 
                  im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                  im.close()
                  im= Image.open(filename)
                  im.load()

                  if index%500==0:
                    print (index)
                  x0=int(row[1]);
                  x1=int(row[2]);
                  y0=int(row[3])
                  y1=int(row[4]);
                  #path=path2+path;
                 # path,x0, y0, x1, y1, _, _ = row
                 # print(index)
                  img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
                  image_height=int(img.shape[0])
                  image_width=int(img.shape[1])
                  self.coords[real_index, 0] = x0 * IMAGE_SIZE / image_width
                  self.coords[real_index, 1] = y0 * IMAGE_SIZE / image_height
                  self.coords[real_index, 2] = (x1 - x0) * IMAGE_SIZE / image_width
                  self.coords[real_index, 3] = (y1 - y0) * IMAGE_SIZE / image_height 

                  self.paths.append(path)
                  real_index+=1;
                except:
                  print (path)
                  
                  continue
        print(len(self.paths))   
        print (real_index)     

    def __len__(self):
        return math.ceil(len(self.paths) / BATCH_SIZE)

    def __getitem__(self, idx):
        batch_paths = self.paths[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]
        batch_coords = self.coords[idx * BATCH_SIZE:(idx + 1) * BATCH_SIZE]

        batch_images = np.zeros((len(batch_paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        for i, f in enumerate(batch_paths):
            img = Image.open(f)
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            img = img.convert('RGB')

            batch_images[i] = preprocess_input(np.array(img, dtype=np.float32))
            img.close()

        return batch_images, batch_coords

In [26]:
import pandas as pd

train_csv=pd.read_csv('C:/Users/itsab/Downloads/training_set.csv')

labels=train_csv.set_index('image_name').T.to_dict('list')

In [27]:
pat=train_csv['image_name']

In [28]:

pat_tr=list(pat)

In [29]:
import cv2
def preprocess_image(img):
  w,h = 224, 224 
  
  img = cv2.resize(img, (w,h))
  img = img/255. 
  
  return img 

In [30]:
import pickle
from random import shuffle
from pathlib import Path
from os import listdir
from os.path import isfile, join
base=Path('images')



shuffle(pat_tr)

def image_generator(input_ids, batch_size = 32):
  
  while True:
    batch_paths = np.random.choice(a= input_ids, size = batch_size)
    
    batch_input = []
    batch_output = []
    
    for input_id in batch_paths:
      input = cv2.imread(join(base, input_id))
      output = labels[input_id]
      
      input = preprocess_image(input)
      
      batch_input += [input]
      batch_output += [output]
   
    batch_x = np.array(batch_input)
    batch_y = np.array(batch_output)
    
    yield (batch_x, batch_y)

In [31]:
train_gen=image_generator(pat_tr,64)

In [32]:
x, y = next(train_gen)

print(x.shape)
print(y.shape)

(64, 224, 224, 3)
(64, 4)


In [33]:
class Validation(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i]
            pred = self.model.predict_on_batch(batch_images)
            mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)

            diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)

            area_gt = gt[:,2] * gt[:,3]
            area_pred = pred[:,2] * pred[:,3]
            union = np.maximum(area_gt + area_pred - intersection, 0)

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        mse = np.round(mse, 4)
        logs["val_mse"] = mse

        print(" - val_iou: {} - val_mse: {}".format(iou, mse))


In [41]:

def create_model(trainable=True):
    #model = MobileNetV2()
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
    # to freeze layers
    for layer in model.layers:
        layer.trainable = trainable

    x = model.layers[-1].output
    x = Conv2D(4, kernel_size=3, name="coords")(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)


In [42]:
model = create_model()

In [36]:
train_datagen = DataGenerator(TRAIN_CSV)

(24001, 4)
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24000
24000


In [17]:
type(train_datagen)

__main__.DataGenerator

In [ ]:
#validation_datagen = Validation(generator=DataGenerator(VALIDATION_CSV))

In [43]:

model.compile(loss="mean_squared_error", optimizer="adam", metrics=[])
checkpoint = ModelCheckpoint("model-{val_iou:.2f}.h5", monitor="val_iou", verbose=1, save_best_only=True,
                               save_weights_only=True, mode="max", period=1)
stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max")
reduce_lr = ReduceLROnPlateau(monitor="val_iou", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")

In [20]:
import numpy as np

len(train_datagen)

import tensorflow as tf
import keras

In [38]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath=model_path, monitor='val_loss', save_best_only=True, verbose=1)

model.fit_generator(generator=train_gen,
                      epochs=EPOCHS,
                      callbacks=[cp],
                      workers=THREADS,
                      use_multiprocessing=False,
                      shuffle=False,steps_per_epoch=None,
                      verbose=1)
model.save(model_path)

In [44]:
model.fit_generator(generator=train_datagen,
                      epochs=10,
                      callbacks=[cp],
                      workers=THREADS,
                      use_multiprocessing=False,
                      shuffle=False,steps_per_epoch=None,
                      verbose=1)


Epoch 1/10
750/750 [==============================] - 2656s 4s/step - loss: 75.1662
Epoch 2/10
750/750 [==============================] - 3256s 4s/step - loss: 40.9616
Epoch 3/10
705/750 [===========================>..] - ETA: 3:28 - loss: 36.1676

KeyboardInterrupt: 

In [ ]:
model.save(model_path)

In [ ]:
model_path_2="C:/Users/itsab/Desktop/Flipkart/model_try_2_2.hd5"

model.fit_generator(generator=train_datagen,
                      epochs=20,
                      callbacks=[cp],
                      workers=THREADS,
                      use_multiprocessing=False,
                      shuffle=True,steps_per_epoch=None,
                      verbose=1)
model.save(model_path_2)